# Machine Learning using Spotify's dataset

- 🎯 The goal of this project is to create effective Machine Learning models to predict the popularity of songs on Spotify.

## 1. Exploratory Data Analysis (EDA)

In [1]:
import pandas as pd
